In [1]:
import os
os.environ['KERAS_BACKEND']='theano'
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import keras.optimizers

Using Theano backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 5

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 5

In [4]:
print(K.image_data_format())
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

channels_last


In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, 'train samples')
print(x_test.shape, 'test samples')

x_train = x_train.reshape((x_train.shape[0],) + input_shape)
x_test = x_test.reshape((x_test.shape[0],) + input_shape)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape, 'train samples')
print(x_test.shape, 'test samples')

(60000, 28, 28) train samples
(10000, 28, 28) test samples
(60000, 28, 28, 1) train samples
(10000, 28, 28, 1) test samples


In [6]:
# convert class vectors to binary class matrices

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [7]:
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [8]:
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [9]:
model = Sequential(feature_layers + classification_layers)

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 32)        25632     
_________________________________________________________________
activation_2 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
__________

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.sgd(),
              metrics=['accuracy'])

In [12]:
t = datetime.datetime.now()
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

print('Training time: %s' % (datetime.datetime.now() - t))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 235s - loss: 1.1832 - acc: 0.6078 - val_loss: 0.3364 - val_acc: 0.9040
Epoch 2/5
60000/60000 [==============================] - 243s - loss: 0.4724 - acc: 0.8557 - val_loss: 0.2249 - val_acc: 0.9353
Epoch 3/5
60000/60000 [==============================] - 301s - loss: 0.3526 - acc: 0.8944 - val_loss: 0.1717 - val_acc: 0.9496
Epoch 4/5
60000/60000 [==============================] - 2010s - loss: 0.2848 - acc: 0.9152 - val_loss: 0.1413 - val_acc: 0.9592
Epoch 5/5
60000/60000 [==============================] - 243s - loss: 0.2429 - acc: 0.9289 - val_loss: 0.1202 - val_acc: 0.9643
Training time: 0:51:15.344685
Test score: 0.120170806164
Test accuracy: 0.9643
